<a href="https://colab.research.google.com/github/imabari/musen/blob/main/musen_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import datetime
import pathlib
import urllib.parse

In [2]:
import pandas as pd
import requests

In [3]:
api = {
    # 1:免許情報検索  2: 登録情報検索
    "ST": 1,
    # 詳細情報付加 0:なし 1:あり
    "DA": 1,
    # スタートカウント
    "SC": 1,
    # 取得件数
    "DC": 1,
    # 出力形式 1:CSV 2:JSON 3:XML
    "OF": 1,
    # 無線局の種別
    "OW": "FB_H",
    # 所轄総合通信局
    "IT": "E",
    # 免許人名称/登録人名称
    "NA": "楽天モバイル",
}

In [4]:
df_code = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vSseDxB5f3nS-YQ1NOkuFKZ7rTNfPLHqTKaSag-qaK25EWLcSL0klbFBZm1b6JDKGtHTk6iMUxsXpxt/pub?gid=284869672&single=true&output=csv",
    dtype={"団体コード": int, "都道府県名": str, "郡名": str, "市区町村名": str},
)

df_code["市区町村名"] = df_code["郡名"].fillna("") + df_code["市区町村名"].fillna("")
df_code.drop("郡名", axis=1, inplace=True)

df_code

,団体コード,都道府県名,市区町村名
0,10006,北海道,
1,11002,北海道,札幌市
2,11011,北海道,札幌市中央区
3,11029,北海道,札幌市北区
4,11037,北海道,札幌市東区
...,...,...,...
1958,473618,沖縄県,島尻郡久米島町
1959,473626,沖縄県,島尻郡八重瀬町
1960,473758,沖縄県,宮古郡多良間村
1961,473812,沖縄県,八重山郡竹富町


In [5]:
def fetch_api(parm, pref):

    r = requests.get("https://www.tele.soumu.go.jp/musen/list", parm)
    r.raise_for_status()

    cr = csv.reader(r.text.splitlines(), delimiter=",")
    data = list(cr)

    # 更新日
    latest = datetime.datetime.strptime(data[0][0], "%Y-%m-%d").date()

    # データラングリング

    df0 = pd.DataFrame(data[1:]).dropna(how="all")

    df1 = df0[25].str.strip().str.split(r"\\n", 2, expand=True)

    se = df1.loc[df1[0].str.contains("携帯電話（その他基地局等"), 2]

    df2 = (
        se.str.strip()
        .str.replace(r"\\n", "")
        .str.extractall("(.+?)\(([0-9,]+?)\)")
        .rename(columns={0: "市区町村名", 1: "開設局数"})
        .reset_index(drop=True)
    )

    df2["市区町村名"] = df2["市区町村名"].str.strip()

    df2["開設局数"] = df2["開設局数"].str.strip().str.replace(",", "").astype(int)

    # 都道府県を抽出
    flag = df2["市区町村名"].str.endswith(("都", "道", "府", "県"))

    # 都道府県に移動
    df2["都道府県名"] = df2["市区町村名"].where(flag).fillna(method="ffill")

    df2["市区町村名"] = df2["市区町村名"].mask(flag, "")

    # df2["更新日"] = latest.isoformat()

    df_pref = df_code[df_code["都道府県名"] == pref]

    # 団体コードを付加
    df3 = (
        pd.merge(df_pref, df2, on=["都道府県名", "市区町村名"], how="left")
        .reset_index(drop=True)
        .reindex(["団体コード", "都道府県名", "市区町村名", "開設局数"], axis=1)
    )

    df3["開設局数"] = df3["開設局数"].fillna(0).astype(int)

    df3["団体コード"] = df3["団体コード"].astype("Int64")

    df3.sort_values("団体コード", inplace=True)

    df4 = pd.read_csv("https://imabari.github.io/musen/07_kinki.csv")
    df4["市区町村名"] = df4["市区町村名"].fillna("")

    df5 = (
        pd.merge(df_pref, df4, on=["団体コード", "都道府県名", "市区町村名"], how="left")
        .reset_index(drop=True)
        .reindex(["団体コード", "都道府県名", "市区町村名", "開設局数"], axis=1)
    )

    df5["開設局数"] = df5["開設局数"].fillna(0).astype(int)

    df5 = df5.astype(df3.dtypes)

    return df3, df5

# 都道府県名を入力

In [6]:
pref = "奈良県"

In [7]:
parm = urllib.parse.urlencode(api, encoding="shift-jis")

df1, df2 = fetch_api(parm, pref)

In [8]:
df1.equals(df2)

True

In [9]:
df3 = pd.merge(df2, df1, on=["団体コード", "都道府県名", "市区町村名"], suffixes = ["_前回", "_今回"])

In [10]:
df3["開設局数_差分"] = df3["開設局数_今回"] - df3["開設局数_前回"]

In [11]:
df3.to_csv("output.csv", encoding="utf_8_sig")

In [12]:
df3

,団体コード,都道府県名,市区町村名,開設局数_前回,開設局数_今回,開設局数_差分
0,290009,奈良県,,187,187,0
1,292010,奈良県,奈良市,52,52,0
2,292028,奈良県,大和高田市,13,13,0
3,292036,奈良県,大和郡山市,18,18,0
4,292044,奈良県,天理市,13,13,0
5,292052,奈良県,橿原市,25,25,0
6,292061,奈良県,桜井市,10,10,0
7,292079,奈良県,五條市,10,10,0
8,292087,奈良県,御所市,2,2,0
9,292095,奈良県,生駒市,19,19,0


# ダウンロード

In [13]:
from google.colab import files

In [14]:
files.download("output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>